In [1]:
# wordy

In [61]:
def split_on_digit(q):
    f = 'X'
    fa = ''
    while f.isalpha():
        fa = fa + ' ' + f
        f, q = q.split(maxsplit=1)
    return fa[3:], int(f), q

def calculate(question):
    q = question.replace('?', ' ?')
    _, operand, rest = split_on_digit(q)
    is_continuing = True
    while is_continuing:
        operator, operand_2, rest = split_on_digit(rest)
        if operator == 'plus': 
            operand += operand_2
        elif operator == 'minus':
            operand -= operand_2
        elif operator == 'multiplied by':
            operand *= operand_2
        elif operator == 'divided by': 
            operand /= operand_2
        else:
            raise ValueError
        if rest == '?':
            is_continuing = False
    return operand

In [62]:
question = "What is -333 divided by 3 plus 1 minus -33?"
calculate(question)

-77.0

In [ ]:
# queen attack

In [ ]:
def board(white_position, black_position):
    board = [['_']*8]*8
    for (r, c), ch in zip((white_position, black_position), ('W', 'B')):
        if min(r, c) < 0 or max(r, c) > 7 or white_position==black_position:
            raise ValueError
        rm = board[r][:]
        rm[c] = ch
        board[r] = rm
    return list(map(lambda r: ''.join(r), board))


def can_attack(white_position, black_position):
    wr, wc = white_position
    br, bc = black_position
    if min(wr, wc, br, bc) < 0 or max(wr, wc, br, bc) > 7 or (wr, wc)==(br, bc):
        raise ValueError
    return wr==br or wc==bc or abs(wr-br)==abs(wc-bc)

In [ ]:
board((2, 3), (5, 6))

In [ ]:
# minesweeper

In [ ]:
def star_ct(r, c, inp):
    row_ct = len(inp)
    col_ct = len(inp[0])
    stars = 0
    for ri in [r-1, r, r+1]:
        for ci in [c-1, c, c+1]:
            if 0 <= ri < row_ct and 0 <= ci < col_ct:
                if inp[ri][ci] == '*':
                    stars += 1
    return str(stars) if stars else ' '

def board(input_board_array):
    if len(set(map(len, input_board_array))) > 1:
        raise ValueError('rows are not of equal length')
    out = []
    for r, row in enumerate(input_board_array):
        out_row = []
        for c, val in enumerate(row):
            if val == ' ':
                out_row.append(star_ct(r, c, input_board_array))
            elif val == '*':
                out_row.append('*')
            else:
                raise ValueError('invalid char encountered')
        out.append(''.join(out_row))
    return out

In [ ]:
inp = ["***", "* *", "***"]
board(inp)

In [ ]:
inp = [" * * "]
board(inp)

In [ ]:
inp = [" ",
       "*",
       " ",
       "*",
       " "]
board(inp)

In [ ]:
inp = ["*",
       " ",
       "  ",
       " ",
       "*"]
board(inp)

In [ ]:
len(set(map(len, inp))) > 1

In [ ]:
# check_brackets

In [ ]:
def check_brackets(input_string):
    br_opening = '{[('
    br_closing = '}])'
    br_closes = dict(zip(br_closing, br_opening))
    br_stack = []
    for ch in input_string:
        if ch in br_opening:
            br_stack.append(ch)
        if ch in br_closing:
            if br_stack and br_closes[ch] == br_stack[-1]:
                br_stack.pop()
            else:
                return False
    return br_stack == []
        


check_brackets("[({]})"), check_brackets("([{}({}[])])")